In [1]:
from data_preprocess import DataPreprocessor
import os
import random
# from embeddings_generator import *


# Hyperparameters

# if __name__ == '__main__':

history_length = 12 # N in article
ra_length = 4 # K in article
discount_factor = 0.99 # Gamma in Bellman equation
actor_lr = 0.0001
critic_lr = 0.001
tau = 0.001 # τ in Algorithm 3
batch_size = 64
nb_episodes = 100
nb_rounds = 50
filename_summary = 'summary.txt'
alpha = 0.5 # α (alpha) in Equation (1)
gamma = 0.9 # Γ (Gamma) in Equation (4)
buffer_size = 1000000 # Size of replay memory D in article
fixed_length = True # Fixed memory length

dg = DataPreprocessor(os.path.dirname(os.getcwd())+'/data/ml-100k/u.data', os.path.dirname(os.getcwd())+'/data/ml-100k/u.item')
# dg.write_csv()

In [2]:
dg.write_csv()

In [3]:
def sample_histo_v5(user_histo, pivot_rating=4, nb_states=5):
    prop_histo = user_histo[user_histo['rating'] >= pivot_rating]
    if len(prop_histo) > nb_states:
        user = user_histo['userId'][0]
        initial_state =  prop_histo[0:nb_states]['itemId'].values.tolist()
        user_history =  prop_histo[nb_states:]['itemId'].values.tolist()
    return user, initial_state, user_history

In [4]:
users = []
initial_states = []
user_histories = []

for user_histo in dg.histo:
    try:
        user, init_state, u_history = sample_histo_v5(user_histo)
        users.append(user)
        initial_states.append(init_state)
        user_histories.append(u_history)
    except:
#         print(len(user_histo[user_histo['rating'] >= 4]))
        continue

In [38]:
user_histo = dg.histo[0]
u_df = sample_histo_v6(user_histo)

In [40]:
u_df[0:]

,user,slate,action
0,195,"[241, 268, 285, 305, 339]","[0, 0, 1, 1, 0]"
1,195,"[1021, 250, 256, 1006, 1240]","[1, 0, 0, 1, 0]"
2,195,"[427, 380, 201, 284, 7]","[1, 1, 0, 1, 1]"
3,195,"[115, 654, 110, 152, 172]","[0, 1, 1, 1, 0]"
4,195,"[237, 69, 381, 392, 286]","[1, 0, 1, 1, 0]"


In [32]:
import pandas as pd

def sample_histo_v6(user_histo, pivot_rating=4, nb_states=5):
    binary_ratings = [1 if i>=4 else 0 for i in user_histo['rating']]
    user = user_histo['userId'][0]
    items = user_histo['itemId'].values.tolist()
    users = []
    slates = []
    actions = []
    for i in range(0,len(items),5):
        if i+5 <= len(items):
            slate = items[i:i+5]
            action = binary_ratings[i:i+5]
            slates.append(slate)
            actions.append(action)
            users.append(user)
            
    user_df = pd.DataFrame()
    user_df['user'] = users
    user_df['slate'] = slates
    user_df['action'] = actions
    return user_df

In [42]:
train_data = []
test_data = []
train_test_ratio = 0.9
for user_histo in dg.histo: 
    u_df = sample_histo_v6(user_histo)
    split_point = int(train_test_ratio*len(u_df))
    u_train_df = u_df[0:split_point]
    u_test_df = u_df[split_point:]
    train_data.append(u_train_df)
    test_data.append(u_test_df)
train_df = pd.concat(train_data)
test_df = pd.concat(test_data)

In [44]:
train_df.to_csv('./train_data.csv', index=False)
test_df.to_csv('./test_data.csv', index=False)

In [5]:
import pandas as pd
train_data = pd.DataFrame()
test_data = pd.DataFrame()

In [6]:
train_test_ratio = 0.8

train_data['user'] = users
train_data['state'] = initial_states
train_data['history'] = [u_h[0:int(train_test_ratio*len(u_h))] for u_h in user_histories]

test_data['user'] = users
test_data['state'] = initial_states
test_data['history'] = [u_h[int(train_test_ratio*len(u_h)):] for u_h in user_histories]

# test_data
# train_data.head()

In [8]:
train_data.to_csv('./data/train_data.csv', index=False)
test_data.to_csv('./data/test_data.csv', index=False)

In [48]:
dt = pd.read_csv('./data/train_data.csv')
dt.head()

,user,state,history
0,389,"[302, 127, 100, 50, 475]","[181, 847, 1007, 1, 275, 845, 124, 285, 471, 2..."
1,894,"[242, 302, 258, 262, 879]","[292, 690, 328, 300, 515, 1251, 1379, 534, 333..."
2,384,"[272, 302, 333, 347, 286]","[258, 313, 327, 300, 879, 989, 748, 355, 316, ..."
3,658,"[515, 100, 276, 9, 127]","[1, 408, 475, 257, 50, 7, 117, 471, 467, 171, ..."
4,711,"[258, 286, 181, 50, 475]","[283, 676, 1115, 476, 275, 744, 151, 25, 10, 1..."


In [ ]:
import random
def sample_histo_v1(user_histo, action_ratio=0.8, max_samp_by_user=5,  max_state=100, max_action=50, nb_states=[], nb_actions=[]):
    '''
    For a given historic, make one or multiple sampling.
    If no optional argument given for nb_states and nb_actions, then the sampling
    is random and each sample can have differents size for action and state.
    To normalize sampling we need to give list of the numbers of states and actions
    to be sampled.

    Parameters
    ----------
    user_histo :  DataFrame
                      historic of user
    delimiter :       string, optional
                      delimiter for the csv
    action_ratio :    float, optional
                      ratio form which movies in history will be selected
    max_samp_by_user: int, optional
                      Number max of sample to make by user
    max_state :       int, optional
                      Number max of movies to take for the 'state' column
    max_action :      int, optional
                      Number max of movies to take for the 'action' action
    nb_states :       array(int), optional
                      Numbers of movies to be taken for each sample made on user's historic
    nb_actions :      array(int), optional
                      Numbers of rating to be taken for each sample made on user's historic

    Returns
    -------
    states :         List(String)
                     All the states sampled, format of a sample: itemId&rating
    actions :        List(String)
                     All the actions sampled, format of a sample: itemId&rating


    Notes
    -----
    States must be before(timestamp<) the actions.
    If given, size of nb_states is the numbller of sample by user
    sizes of nb_states and nb_actions must be equals
    '''

    n = len (user_histo)
    sep = int (action_ratio * n)
    nb_sample = random.randint (1, max_samp_by_user)
    if not nb_states:
        nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
    if not nb_actions:
        nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
    assert len (nb_states) == len (nb_actions), 'Given array must have the same size'

    states = []
    actions = []
    # SELECT SAMPLES IN HISTO
    for i in range (len (nb_states)):
        sample_states = user_histo.iloc[0:sep].sample (nb_states[i])
        sample_actions = user_histo.iloc[-(n - sep):].sample (nb_actions[i])

        sample_state = []
        sample_action = []
        for j in range (nb_states[i]):
            row = sample_states.iloc[j]
            # FORMAT STATE
            state = str (row.loc['itemId']) + '&' + str (row.loc['rating'])
            sample_state.append (state)

        for j in range (nb_actions[i]):
            row = sample_actions.iloc[j]
            # FORMAT ACTION
            action = str (row.loc['itemId']) + '&' + str (row.loc['rating'])
            sample_action.append (action)


        states.append (sample_state)
        actions.append (sample_action)

    return states, actions

In [ ]:
import random
def sample_histo_v2(user_histo, action_ratio=0.8, max_samp_by_user=5,  max_state=100, max_action=50, nb_states=[], nb_actions=[]):
    '''
    For a given historic, make one or multiple sampling.
    If no optional argument given for nb_states and nb_actions, then the sampling
    is random and each sample can have differents size for action and state.
    To normalize sampling we need to give list of the numbers of states and actions
    to be sampled.

    Parameters
    ----------
    user_histo :  DataFrame
                      historic of user
    delimiter :       string, optional
                      delimiter for the csv
    action_ratio :    float, optional
                      ratio form which movies in history will be selected
    max_samp_by_user: int, optional
                      Number max of sample to make by user
    max_state :       int, optional
                      Number max of movies to take for the 'state' column
    max_action :      int, optional
                      Number max of movies to take for the 'action' action
    nb_states :       array(int), optional
                      Numbers of movies to be taken for each sample made on user's historic
    nb_actions :      array(int), optional
                      Numbers of rating to be taken for each sample made on user's historic

    Returns
    -------
    states :         List(String)
                     All the states sampled, format of a sample: itemId&rating
    actions :        List(String)
                     All the actions sampled, format of a sample: itemId&rating


    Notes
    -----
    States must be before(timestamp<) the actions.
    If given, size of nb_states is the numbller of sample by user
    sizes of nb_states and nb_actions must be equals
    '''

    n = len(user_histo)
    sep = int (action_ratio * n)
    nb_sample = random.randint (1, max_samp_by_user)
    if not nb_states:
        nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
    if not nb_actions:
        nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
    assert len (nb_states) == len (nb_actions), 'Given array must have the same size'
    
    states = []
    actions = []
    for i in range (len (nb_states)):
        user_states = user_histo[user_histo['rating']>=4]
        if len(user_states) >= nb_states[i]:
            sample_states = user_states.sample (nb_states[i])
        else:
            continue

        sample_actions = user_histo.iloc[-(n - sep):].sample (nb_actions[i])

        sample_state = []
        sample_action = []

        for j in range (nb_states[i]):
            row = sample_states.iloc[j]
            # FORMAT STATE
            state = str (row.loc['itemId']) + '&' + str (1)
            sample_state.append (state)

        for k in range (nb_actions[i]):
            row = sample_actions.iloc[k]
            # FORMAT ACTION
            if row.loc['rating'] >= 4:
                action = str (row.loc['itemId']) + '&' + str (1)
            else:
                action = str (row.loc['itemId']) + '&' + str (0)
            sample_action.append (action)

        states.append (sample_state)
        actions.append (sample_action)
    return states, actions

In [ ]:
import random
import copy
def sample_histo_v3(user_histo, action_ratio=0.8, max_samp_by_user=5,  max_state=100, max_action=50, nb_states=[], nb_actions=[]):
    '''
    For a given historic, make one or multiple sampling.
    If no optional argument given for nb_states and nb_actions, then the sampling
    is random and each sample can have differents size for action and state.
    To normalize sampling we need to give list of the numbers of states and actions
    to be sampled.

    Parameters
    ----------
    user_histo :  DataFrame
                      historic of user
    delimiter :       string, optional
                      delimiter for the csv
    action_ratio :    float, optional
                      ratio form which movies in history will be selected
    max_samp_by_user: int, optional
                      Number max of sample to make by user
    max_state :       int, optional
                      Number max of movies to take for the 'state' column
    max_action :      int, optional
                      Number max of movies to take for the 'action' action
    nb_states :       array(int), optional
                      Numbers of movies to be taken for each sample made on user's historic
    nb_actions :      array(int), optional
                      Numbers of rating to be taken for each sample made on user's historic

    Returns
    -------
    states :         List(String)
                     All the states sampled, format of a sample: itemId&rating
    actions :        List(String)
                     All the actions sampled, format of a sample: itemId&rating


    Notes
    -----
    States must be before(timestamp<) the actions.
    If given, size of nb_states is the numbller of sample by user
    sizes of nb_states and nb_actions must be equals
    '''

    n = len(user_histo)
    sep = int (action_ratio * n)
    nb_sample = random.randint (1, max_samp_by_user)
    if not nb_states:
        nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
    if not nb_actions:
        nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
    assert len (nb_states) == len (nb_actions), 'Given array must have the same size'
    
    states = []
    for n in range(len(nb_states)):
        state_len = nb_states[n]
        action_len = nb_actions[n]

        item_list = user_histo['itemId'].values.tolist()
        click_list = user_histo['rating'].values.tolist()
        initial_state = []
        initial_end = 0
        for i in range(len(item_list)):
            if click_list[i] >= 4 and len(initial_state) < state_len:
                initial_state.append(item_list[i])
                initial_end = i
        


        if len(initial_state) == state_len and (initial_end + action_len <= len(item_list)):
            current_state = copy.copy(initial_state)
            for i in range(initial_end+1,len(item_list),action_len):
                if i+action_len <= len(item_list):
                    actions.append([str(item) + '&' + str (1) if rating >= 4 else str(item) + '&' + str (0) for item,rating in zip(item_list[i:i+action_len],click_list[i:i+action_len])])
                    rewards.append(click_list[i:i+action_len])
                    states.append([str(i) + '&' + str (1) for i in copy.copy(current_state)])
                    for j in range(i,i+action_len):
                        if click_list[j] == 1:
                            current_state.append(item_list[j])
                            del current_state[0]
            return states, actions

In [4]:
import random
import copy
def sample_histo_v4(user_histo, action_ratio=0.8, max_samp_by_user=5,  max_state=100, max_action=50, nb_states=[], nb_actions=[]):
    '''
    For a given historic, make one or multiple sampling.
    If no optional argument given for nb_states and nb_actions, then the sampling
    is random and each sample can have differents size for action and state.
    To normalize sampling we need to give list of the numbers of states and actions
    to be sampled.

    Parameters
    ----------
    user_histo :  DataFrame
                      historic of user
    delimiter :       string, optional
                      delimiter for the csv
    action_ratio :    float, optional
                      ratio form which movies in history will be selected
    max_samp_by_user: int, optional
                      Number max of sample to make by user
    max_state :       int, optional
                      Number max of movies to take for the 'state' column
    max_action :      int, optional
                      Number max of movies to take for the 'action' action
    nb_states :       array(int), optional
                      Numbers of movies to be taken for each sample made on user's historic
    nb_actions :      array(int), optional
                      Numbers of rating to be taken for each sample made on user's historic

    Returns
    -------
    states :         List(String)
                     All the states sampled, format of a sample: itemId&rating
    actions :        List(String)
                     All the actions sampled, format of a sample: itemId&rating


    Notes
    -----
    States must be before(timestamp<) the actions.
    If given, size of nb_states is the numbller of sample by user
    sizes of nb_states and nb_actions must be equals
    '''

    n = len(user_histo)
    sep = int (action_ratio * n)
    nb_sample = random.randint (1, max_samp_by_user)
    if not nb_states:
        nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
    if not nb_actions:
        nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
    assert len (nb_states) == len (nb_actions), 'Given array must have the same size'
    
    states = []
    actions = []
    rewards = []
    
    user = [[str(int(user_histo['userId'].values.tolist()[0])) + '&' + str (1)]]
    
    for n in range(len(nb_states)):
        state_len = nb_states[n]
        action_len = nb_actions[n]

        item_list = user_histo['itemId'].values.tolist()
        click_list = user_histo['rating'].values.tolist()
        initial_state = []
        initial_end = 0
        for i in range(len(item_list)):
            if click_list[i] >= 4 and len(initial_state) < state_len:
                initial_state.append(item_list[i])
                initial_end = i



        if len(initial_state) == state_len:
            states.append([str(i) + '&' + str (1) for i in initial_state])
            return user,states

In [ ]:
user_histo = dg.train[0]
action_ratio=0.8
max_samp_by_user=5
max_state=100
max_action=50
nb_states=[12]
nb_actions=[4]

In [ ]:
n = len (user_histo)
sep = int (action_ratio * n)
nb_sample = random.randint (1, max_samp_by_user)
n,sep,nb_sample

In [ ]:
if not nb_states:
    nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
if not nb_actions:
    nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
assert len (nb_states) == len (nb_actions), 'Given array must have the same size'

In [8]:
for user_histo in dg.histo:
    states, actions = sample_histo_v4(user_histo,nb_states=[5],nb_actions=[5])
    print(states)
    print(actions)
# #     break
#     for i in range(len(states)):
#         # FORMAT STATE
#         state_str = '|'.join (states[i])
#         print(state_str)
# #         FORMAT ACTION
#         action_str = '|'.join (actions[i])
#         print(action_str)
#         # FORMAT N_STATE
#         n_state_str = state_str + '|' + action_str
#         print(n_state_str)
#         break
#     break

[['195&1']]
[['285&1', '305&1', '1021&1', '1006&1', '427&1']]
[['185&1']]
[['299&1', '321&1', '590&1', '290&1', '99&1']]
[['21&1']]
[['257&1', '185&1', '172&1', '152&1', '237&1']]
[['243&1']]
[['267&1', '885&1', '323&1', '257&1', '293&1']]
[['165&1']]
[['312&1', '257&1', '346&1', '750&1', '327&1']]
[['297&1']]
[['285&1', '171&1', '587&1', '173&1', '68&1']]
[['114&1']]
[['301&1', '136&1', '627&1', '272&1', '695&1']]
[['252&1']]
[['299&1', '688&1', '327&1', '342&1', '293&1']]
[['304&1']]
[['285&1', '241&1', '689&1', '301&1', '903&1']]
[['5&1']]
[['241&1', '268&1', '301&1', '305&1', '303&1']]
[['61&1']]
[['257&1', '305&1', '874&1', '474&1', '272&1']]
[['285&1']]
[['288&1', '287&1', '407&1', '247&1', '150&1']]
[['199&1']]
[['303&1', '257&1', '324&1', '124&1', '275&1']]
[['209&1']]
[['299&1', '22&1', '57&1', '968&1', '55&1']]
[['223&1']]
[['312&1', '299&1', '327&1', '293&1', '325&1']]
[['302&1']]
[['257&1', '285&1', '301&1', '268&1', '287&1']]
[['121&1']]
[['268&1', '27&1', '512&1', '174&1'

TypeError: 'NoneType' object is not iterable

In [ ]:
for user_histo in dg.train:
    states, actions = sample_histo_v2(user_histo,nb_states=[12],nb_actions=[4])
#     print(states)
#     print(actions)
#     break
    for i in range (len (states)):
        # FORMAT STATE
        state_str = '|'.join (states[i])
        print(state_str)
        # FORMAT ACTION
        action_str = '|'.join (actions[i])
        print(action_str)
        # FORMAT N_STATE
        n_state_str = state_str + '|' + action_str
        print(n_state_str)
        break
    break

In [ ]:
for user_histo in dg.train:
    states, actions = sample_histo_v3(user_histo,nb_states=[12],nb_actions=[4])
    print(len(states))
#     print(actions)
#     break
    for i in range (len (states)):
        # FORMAT STATE
        state_str = '|'.join (states[i])
#         print(state_str)
        # FORMAT ACTION
        action_str = '|'.join (actions[i])
#         print(action_str)
        # FORMAT N_STATE
        n_state_str = state_str + '|' + action_str
#         print(n_state_str)
        
        

In [29]:
import pandas as pd
import numpy as np
def read_file(data_path):
    ''' Load data from train.csv or test.csv. '''

    data = pd.read_csv (data_path, sep = ';')
    for col in ['user', 'state']:
        data[col] = [np.array ([[np.int(k) for k in ee.split ('&')] for ee in e.split ('|')]) for e in data[col]]
    for col in ['user', 'state']:
        data[col] = [np.array ([e[0] for e in l]) for l in data[col]]
        
    return data

In [ ]:
data = pd.read_csv ('train.csv', sep = ';')
for col in ['state', 'n_state', 'action_reward']:
    data[col] = [np.array ([[np.int (k) for k in ee.split ('&')] for ee in e.split ('|')]) for e in data[col]]
for col in ['state', 'n_state']:
    data[col] = [np.array ([e[0] for e in l]) for l in data[col]]




In [ ]:
data['state'] = [[e[0] for e in l] for l in data['action_reward']]

data.drop (columns = ['action_reward'], inplace = True)

In [35]:
data = read_file('train.csv')

In [36]:
data.head()

,user,state
0,[389],"[302, 127, 100, 50, 475, 181, 847, 1007, 1, 27..."
1,[894],"[242, 302, 258, 262, 879, 292, 690, 328, 300, ..."
2,[384],"[272, 302, 333, 347, 286, 258, 313, 327, 300, ..."
3,[658],"[515, 100, 276, 9, 127, 1, 408, 475, 257, 50, ..."
4,[711],"[258, 286, 181, 50, 475, 283, 676, 1115, 476, ..."


In [40]:
import copy
data['n_state'] = pd.concat([copy.copy(data['user']),copy.copy(data['state'])])

ValueError: cannot reindex from a duplicate axis

In [34]:
data.head()

,user,state,n_state
0,[389],"[302, 127, 100, 50, 475, 181, 847, 1007, 1, 27...","[691, 516, 489, 439, 864, 570, 1236, 1396, 390..."
1,[894],"[242, 302, 258, 262, 879, 292, 690, 328, 300, ...","[1136, 1196, 1152, 1156, 1773, 1186, 1584, 122..."
2,[384],"[272, 302, 333, 347, 286, 258, 313, 327, 300, ...","[656, 686, 717, 731, 670, 642, 697, 711, 684, ..."
3,[658],"[515, 100, 276, 9, 127, 1, 408, 475, 257, 50, ...","[1173, 758, 934, 667, 785, 659, 1066, 1133, 91..."
4,[711],"[258, 286, 181, 50, 475, 283, 676, 1115, 476, ...","[969, 997, 892, 761, 1186, 994, 1387, 1826, 11..."
